Downloading grammar_ru to your google drive

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [1]:
%cd gdrive/MyDrive/Colab

/content/gdrive/MyDrive/Colab


In [ ]:
!git clone https://github.com/okulovsky/grammar_ru.git


In [2]:
%cd grammar_ru

/content/gdrive/MyDrive/Colab/grammar_ru


In [66]:
!git status

On branch DatasetPipeline
Your branch is up to date with 'origin/DatasetPipeline'.

Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	deleted:    research/InfinitiveOrFinite/Data/X_test.csv
	deleted:    research/InfinitiveOrFinite/Data/X_train.csv
	deleted:    research/InfinitiveOrFinite/Data/Y_test.csv
	deleted:    research/InfinitiveOrFinite/Data/Y_train.csv
	deleted:    research/InfinitiveOrFinite/InfinitiveOrFinite.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	research/infinitive_or_finite/
	research/pipeline_test/

no changes added to commit (use "git add" and/or "git commit -a")


Installing dependencies

In [ ]:
!python setup.py install

In [ ]:
!sudo bash setup.sh

(Probably should restart environment there to apply newly installed packages)

In [ ]:
! sudo apt-get install libenchant1c2a

Creating text dataframe

In [3]:
from grammar_ru.common import natasha
import pandas as pd
import numpy as np

df = natasha.make_dataframe_from_nerus(10)
df

,word_id,sentence_id,word_index,word_offset,word,word_type,word_length,paragraph_id
0,0,0,0,0,Вице-премьер,ru,12,0
1,1,0,1,13,по,ru,2,0
2,2,0,2,16,социальным,ru,10,0
3,3,0,3,27,вопросам,ru,8,0
4,4,0,4,36,Татьяна,ru,7,0
...,...,...,...,...,...,...,...,...
2035,2035,116,12,1651,произошли,ru,9,9
2036,2036,116,13,1661,при,ru,3,9
2037,2037,116,14,1665,ремонте,ru,7,9
2038,2038,116,15,1673,крейсера,ru,8,9


We can either pack everything into single dataframe using NlpAnalyzationPipeline like this:

In [5]:
from grammar_ru.common.architecture.nlp_analyzation_pipeline import NlpAnalyzationPipeline 
from grammar_ru.analyzers.natasha.natasha_morph_analyzer import NatashaMorphAnalyzer
from grammar_ru.analyzers.natasha.natasha_syntax_analyzer import NatashaSyntaxAnalyzer

morph = NatashaMorphAnalyzer()
syntax = NatashaSyntaxAnalyzer()

pipeline = NlpAnalyzationPipeline([("Morph", morph), ("Syntax", syntax)])

analyzed = pipeline.analyze_dataframe(df)
analyzed

,word_id,sentence_id,word_index,word_offset,word,word_type,word_length,paragraph_id,Morph_word_id,Morph_POS,Morph_Animacy,Morph_Case,Morph_Gender,Morph_Number,Morph_Degree,Morph_Aspect,Morph_Mood,Morph_Tense,Morph_VerbForm,Morph_Voice,Morph_Variant,Morph_Person,Morph_Polarity,Morph_Foreign,Syntax_parent_id,Syntax_rel,Syntax_word_id
0,0,0,0,0,Вице-премьер,ru,12,0,0,NOUN,Anim,Nom,Masc,Sing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,nsubj,0
1,1,0,1,13,по,ru,2,0,1,ADP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,case,1
2,2,0,2,16,социальным,ru,10,0,2,ADJ,NaN,Dat,NaN,Plur,Pos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,amod,2
3,3,0,3,27,вопросам,ru,8,0,3,NOUN,Inan,Dat,Masc,Plur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,nmod,3
4,4,0,4,36,Татьяна,ru,7,0,4,PROPN,Anim,Nom,Fem,Sing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,appos,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2035,2035,116,12,1651,произошли,ru,9,9,2035,VERB,NaN,NaN,NaN,Plur,NaN,Perf,Ind,Past,Fin,Act,NaN,NaN,NaN,NaN,2032,acl:relcl,2035
2036,2036,116,13,1661,при,ru,3,9,2036,ADP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2037,case,2036
2037,2037,116,14,1665,ремонте,ru,7,9,2037,NOUN,Inan,Loc,Masc,Sing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2035,obl,2037
2038,2038,116,15,1673,крейсера,ru,8,9,2038,NOUN,Inan,Gen,Masc,Sing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2037,nmod,2038


Or we can use analyzers independently in order to keep data in different dataframes

In [6]:
morph_analyzed = morph.analyze(df)
morph_analyzed

,word_id,POS,Animacy,Case,Gender,Number,Degree,Aspect,Mood,Tense,VerbForm,Voice,Variant,Person,Polarity,Foreign
0,0,NOUN,Anim,Nom,Masc,Sing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,ADP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,ADJ,NaN,Dat,NaN,Plur,Pos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NOUN,Inan,Dat,Masc,Plur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,PROPN,Anim,Nom,Fem,Sing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2035,2035,VERB,NaN,NaN,NaN,Plur,NaN,Perf,Ind,Past,Fin,Act,NaN,NaN,NaN,NaN
2036,2036,ADP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2037,2037,NOUN,Inan,Loc,Masc,Sing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2038,2038,NOUN,Inan,Gen,Masc,Sing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
syntax_analyzed = syntax.analyze(df)
syntax_analyzed

,parent_id,rel,word_id
0,6,nsubj,0
1,3,case,1
2,3,amod,2
3,0,nmod,3
4,0,appos,4
...,...,...,...
2035,2032,acl:relcl,2035
2036,2037,case,2036
2037,2035,obl,2037
2038,2037,nmod,2038


Save your dataframes as you wish, and then begin batched training process using TG

In [27]:
from tg.common.ml import batched_training as bt

def create_index_frame(df):
  return df[(df["word"].str.endswith("тся")) | (df["word"].str.endswith("ться"))]

index_frame = create_index_frame(df).set_index('word_id')
index_frame

,sentence_id,word_index,word_offset,word,word_type,word_length,paragraph_id
word_id,,,,,,,
228,14,16,823,тренируется,ru,11,1
290,19,2,153,сообщается,ru,10,2
442,28,2,200,содержатся,ru,10,3
728,43,10,759,ссылаются,ru,9,4
895,51,6,1942,финансируется,ru,13,4
956,54,2,275,говорится,ru,9,5
1522,82,13,1039,пошевелиться,ru,12,7
1772,102,10,833,является,ru,8,8
2013,115,5,1525,используется,ru,12,9


In this example, for each word we extract its parents, brothers and children

In [47]:
from grammar_ru.extractors.slovnet_context_extractor import ContextExtractor

bundle = bt.DataBundle(index_frame, dict(
    syntax=syntax_analyzed,
    morph=morph_analyzed
))

context_extractor = ContextExtractor("Context", max_shift=1)

extracted_context = context_extractor.extract(index_frame, bundle)
extracted_context

{'key': 'Context', 'value':      word_id  shift  relative_word_id
0        228      1               224
1        228      0               223
2        228      0               228
3        228     -1               225
4        228     -1               226
..       ...    ...               ...
651     2013      0              2027
652     2013     -1              2008
653     2013     -1              2012
654     2013     -1              2017
655     2013     -1              2022

[656 rows x 3 columns]}

We can alternatively pack this into batcher.

In [49]:
batcher = bt.Batcher(
    batch_size = 3,
    extractors = [context_extractor],
    batching_strategy = bt.SimpleBatcherStrategy()
)
batcher.fit_extractors(bundle)
batcher.get_batch(bt.IndexedDataBundle(bundle, bundle.index_frame.index),0)

{'Context':      word_id  shift  relative_word_id
 0        228      1               224
 1        228      0               223
 2        228      0               228
 3        228     -1               225
 4        228     -1               226
 ..       ...    ...               ...
 253      442      0              2013
 254      442      0              2027
 255      442     -1               441
 256      442     -1               443
 257      442     -1               454
 
 [258 rows x 3 columns],
 'index':          sentence_id  word_index  ...  word_length paragraph_id
 word_id                           ...                          
 228               14          16  ...           11            1
 290               19           2  ...           10            2
 442               28           2  ...           10            3
 
 [3 rows x 7 columns]}

Let's test performance

In [56]:
%%time

new_df = natasha.make_dataframe_from_nerus(1000)

CPU times: user 8.58 s, sys: 160 ms, total: 8.74 s
Wall time: 8.6 s


In [57]:
new_df

,word_id,sentence_id,word_index,word_offset,word,word_type,word_length,paragraph_id
0,0,0,0,0,Вице-премьер,ru,12,0
1,1,0,1,13,по,ru,2,0
2,2,0,2,16,социальным,ru,10,0
3,3,0,3,27,вопросам,ru,8,0
4,4,0,4,36,Татьяна,ru,7,0
...,...,...,...,...,...,...,...,...
212350,212350,11644,13,768,их,ru,2,999
212351,212351,11644,14,771,другому,ru,7,999
212352,212352,11644,15,779,человеку,ru,8,999
212353,212353,11644,16,788,невозможно,ru,10,999


In [58]:
%%time

new_morph_analyzed = morph.analyze(new_df)

CPU times: user 48.3 s, sys: 16.7 s, total: 1min 4s
Wall time: 42.2 s


In [59]:
new_morph_analyzed

,word_id,POS,Animacy,Case,Gender,Number,Degree,Aspect,Mood,Tense,VerbForm,Voice,Variant,Person,Polarity,Foreign
0,0,NOUN,Anim,Nom,Masc,Sing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,ADP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,ADJ,NaN,Dat,NaN,Plur,Pos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NOUN,Inan,Dat,Masc,Plur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,PROPN,Anim,Nom,Fem,Sing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212350,212350,PRON,Inan,Acc,NaN,Plur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN
212351,212351,ADJ,NaN,Dat,Masc,Sing,Pos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
212352,212352,NOUN,Anim,Dat,Masc,Sing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
212353,212353,ADJ,NaN,NaN,Neut,Sing,Pos,NaN,NaN,NaN,NaN,NaN,Short,NaN,NaN,NaN


In [60]:
%%time

new_syntax_analyzed = syntax.analyze(new_df)

CPU times: user 7min 42s, sys: 2min 46s, total: 10min 29s
Wall time: 7min 33s


In [61]:
new_syntax_analyzed

,parent_id,rel,word_id
0,6,nsubj,0
1,3,case,1
2,3,amod,2
3,0,nmod,3
4,0,appos,4
...,...,...,...
212350,212349,obj,212350
212351,212352,amod,212351
212352,212353,iobj,212352
212353,212344,conj,212353


In [62]:
%%time

new_index_frame = create_index_frame(new_df).set_index("word_id")

CPU times: user 279 ms, sys: 187 µs, total: 279 ms
Wall time: 281 ms


In [65]:
new_index_frame

,sentence_id,word_index,word_offset,word,word_type,word_length,paragraph_id
word_id,,,,,,,
228,14,16,823,тренируется,ru,11,1
290,19,2,153,сообщается,ru,10,2
442,28,2,200,содержатся,ru,10,3
728,43,10,759,ссылаются,ru,9,4
895,51,6,1942,финансируется,ru,13,4
...,...,...,...,...,...,...,...
212138,11632,12,929,наблюдается,ru,11,998
212165,11633,13,1128,прятаться,ru,9,998
212171,11634,3,1173,продолжаются,ru,12,998


In [63]:
%%time

new_bundle = bt.DataBundle(new_index_frame, dict(
    syntax=new_syntax_analyzed,
    morph=new_morph_analyzed
))

extracted_context = context_extractor.extract(new_index_frame, new_bundle)

CPU times: user 15min 53s, sys: 14.1 s, total: 16min 7s
Wall time: 16min 8s


In [64]:
extracted_context

{'key': 'Context', 'value':           word_id  shift  relative_word_id
0             228      1               224
1             228      0               223
2             228      0               228
3             228     -1               225
4             228     -1               226
...           ...    ...               ...
11885605   212344     -1            212340
11885606   212344     -1            212341
11885607   212344     -1            212343
11885608   212344     -1            212345
11885609   212344     -1            212353

[11885610 rows x 3 columns]}

We see, that this code works not very fast. 

It can be optimized in two places:

1. SyntaxAnalyzer

  This part works slowly because of not optimized translation from relative parent id to absolute parent id

2. ContextExtractor

  This part works slowly for similar reasons, and it also uses iterrows which is slow method by design. I assume that context extraction code may be rewritten in much faster way, but this requires additional analyzation.